In [0]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [0]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
import urllib.parse
import re
from tqdm import tqdm_notebook

In [0]:
options = webdriver.ChromeOptions()
options.add_argument ('--headless')
options.add_argument ('--no-sandbox')
options.add_argument ('--disable-dev-shm-usage')

browser = webdriver.Chrome ( 'chromedriver' , options = options)

In [0]:
# 검색을 위한 변수 설정 
startdate=dt.date(year=2018,month=1,day=1)
untildate=dt.date(year=2018,month=1,day=2)
enddate = dt.date(year=2018,month=12,day=31)
keyword = urllib.parse.quote_plus("랄라블라")

In [0]:
# tweet_bag=[]
tweet = []
daily_tweet = {}

In [0]:
while not enddate==startdate:
  
  print(startdate)
  daily_tweet[str(startdate)] = []

  url='https://twitter.com/search?l=ko&q='+keyword+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'

  browser.get (url)

  html = browser.page_source
  soup=BeautifulSoup(html,'html.parser')

  lastHeight = browser.execute_script("return document.body.scrollHeight")

  while True:
    tweets=soup.find_all("p", {"class": "TweetTextSize"})

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    newHeight = browser.execute_script("return document.body.scrollHeight")
    
    if newHeight != lastHeight:
#       tweet_bag.append(tweets)
      daily_tweet[str(startdate)] += tweets
    else:
      startdate=untildate
      untildate+=dt.timedelta(days=1)
      break
  
    lastHeight = newHeight

In [0]:
len(daily_tweet.keys())

In [0]:
def cleantweet(total_tweets_dict):
  keys = list(total_tweets_dict.keys())
  cleaned_tweet_dict = {}
  
  for key in tqdm_notebook(keys,desc = '일별할당'):
    cleaned_tweet = list()
    cleaned_tweet_dict[key] = []
    
    tweets_value = total_tweets_dict[key]
    
    for tweet in tqdm_notebook(tweets_value,desc = '트윗 클렌징'):
      tmp = re.sub('<.+?>', '', str(tweet), 0).strip()
      cleaned_tweet.append(tmp)
    
    cleaned_tweet_dict[key] += cleaned_tweet
    
  return cleaned_tweet_dict

In [0]:
cleaned_tweet_dict = cleantweet(daily_tweet)

In [0]:
# import pickle

# with open('2018_lalavla.txt', 'wb') as f:
#      pickle.dump(cleaned_tweet_bag, f)

In [0]:
# with open('2018_lalavla.txt', 'rb') as f:
# ...     data = pickle.load(f) # 단 한줄씩 읽어옴